In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
data = pd.read_csv('StudentsPerformance.csv', delimiter=',')
data = data.drop(columns=['race/ethnicity', 'parental level of education', 'lunch', 'test preparation course'])
data.loc[data["gender"] == "male", "gender"] = 1
data.loc[data["gender"] == "female", "gender"] = 0
data.dropna()
x=np.array(data[['math score', 'reading score', 'writing score']])
y=np.array(data['gender'])
pca=PCA(n_components=2)
x_2=pca.fit_transform(x)

Векторизуємо датасет

In [3]:
all_features = []
all_targets = []
with open('StudentsPerformance.csv') as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([int(0 if v.replace('"', "") == "female" else 1) for v in fields[:-1]])
        all_targets.append([int(0 if fields[0].replace('"', "")== "female" else 1)])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "gender","race/ethnicity","parental level of education","lunch","test preparation course","math score","reading score","writing score"
EXAMPLE FEATURES: [0, 1, 1, 1, 1, 1, 1]
features.shape: (1000, 7)
targets.shape: (1000, 1)


Підготовка тестового датасету

In [4]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 800
Number of validation samples: 200


In [5]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 387 (48.38% of total)


In [6]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

<ipython-input-6-c2c1e75ceb8d>:5: RuntimeWarning: invalid value encountered in divide
  train_features /= std
<ipython-input-6-c2c1e75ceb8d>:6: RuntimeWarning: invalid value encountered in divide
  val_features /= std


In [7]:
import keras

model = keras.Sequential(
    [
        keras.Input(shape=train_features.shape[1:]),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               2048      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 133889 (523.00 KB)
Trainable params: 13388

In [8]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"]
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
1/1 - 3s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 3s/epoch - 3s/step
Epoch 2/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 130ms/epoch - 130ms/step
Epoch 3/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 263ms/epoch - 263ms/step
Epoch 4/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 195ms/epoch - 195ms/step
Epoch 5/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 147ms/epoch - 147ms/step
Epoch 6/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 107ms/epoch - 107ms/step
Epoch 7/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 129ms/epoch - 129ms/step
Epoch 8/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 151ms/epoch - 151ms/step
Epoch 9/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 241ms/epoch - 241ms/step
Epoch 10/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_accuracy: 0.5250 - 95ms/epoch - 95ms/step
Epoch 11/30
1/1 - 0s - loss: nan - accuracy: 0.5163 - val_loss: nan - val_ac

2. Згорткові нейронні мережі

In [10]:
zip_ref = zipfile.ZipFile('/content/TRAIN.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content/sample_data') #Extracts the files into the /tmp folder
zip_ref.close()

In [11]:
import keras
from keras import layers
import zipfile
import os
import tensorflow as tf
import matplotlib.pyplot as plt



num_skipped = 0
for folder_name in ("EOSINOPHIL", "LYMPHOCYTE", "MONOCYTE", "NEUTROPHIL"):
    folder_path = os.path.join("/content/sample_data/TRAIN", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [12]:
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "/content/sample_data/TRAIN",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 9957 files belonging to 4 classes.
Using 7966 files for training.
Using 1991 files for validation.


In [13]:

data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal", input_shape=(180, 180, 3)),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
  ])

In [14]:
AUTOTUNE = tf.data.AUTOTUNE

train = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [15]:
model_2 = keras.Sequential([
  data_augmentation,
  keras.layers.Rescaling(1./255),
  keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='tanh'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Dropout(0.4),
  keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='tanh'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Flatten(),
  keras.layers.Dense(256, activation='softmax'),
  keras.layers.Dense(5, name="outputs")
])

In [16]:

model_2.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x7948c8990c40>>

In [17]:

model_2.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0002), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model_2.fit(train, validation_data = val_ds, epochs=10)

Epoch 1/10
63/63 [==============================] - 321s 5s/step - loss: 1.5766 - accuracy: 0.2469 - val_loss: 1.6029 - val_accuracy: 0.2506
Epoch 2/10
63/63 [==============================] - 298s 5s/step - loss: 1.5658 - accuracy: 0.2503 - val_loss: 1.5981 - val_accuracy: 0.2612
Epoch 3/10
63/63 [==============================] - 280s 4s/step - loss: 1.5468 - accuracy: 0.2801 - val_loss: 1.5884 - val_accuracy: 0.2682
Epoch 4/10
63/63 [==============================] - 283s 5s/step - loss: 1.5361 - accuracy: 0.3131 - val_loss: 1.5716 - val_accuracy: 0.3154
Epoch 5/10
63/63 [==============================] - 286s 5s/step - loss: 1.5230 - accuracy: 0.3791 - val_loss: 1.5496 - val_accuracy: 0.3069
Epoch 6/10
63/63 [==============================] - 278s 4s/step - loss: 1.5078 - accuracy: 0.4215 - val_loss: 1.5411 - val_accuracy: 0.2496
Epoch 7/10
63/63 [==============================] - 284s 5s/step - loss: 1.4957 - accuracy: 0.4326 - val_loss: 1.5391 - val_accuracy: 0.2506
Epoch 8/10
63

Рекурентні нейромережі

In [34]:
import numpy as np
import keras
from keras import layers

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

In [35]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 128)         2560000   
                                                                 
 bidirectional_2 (Bidirecti  (None, None, 128)         98816     
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2757761 (10.52 MB)
Trainable params: 2757761 

In [36]:
df1 = pd.read_csv('test.csv')
df1

,text,label
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
...,...,...
1995,i just keep feeling like someone is being unki...,3
1996,im feeling a little cranky negative after this...,3
1997,i feel that i am useful to my people and that ...,1
1998,im feeling more comfortable with derby i feel ...,1


In [37]:
df1=df1[0:5000]
from collections import Counter
Counter( df1['label'])


Counter({0: 581, 1: 695, 4: 224, 3: 275, 2: 159, 5: 66})

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df1['text'], df1['label'], test_size=0.20)

In [39]:
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 5000
oov_token = ""
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)



In [41]:

x_train_sequences = tokenizer.texts_to_sequences(X_train)
x_test_sequences = tokenizer.texts_to_sequences(X_test)

In [42]:
max_length = 100
padding_type = "post"
trunction_type="post"
x_train_padded = keras.utils.pad_sequences(x_train_sequences,maxlen=max_length, padding=padding_type,
                       truncating=trunction_type)
x_test_padded = keras.utils.pad_sequences(x_test_sequences,maxlen=max_length,
                               padding=padding_type, truncating=trunction_type)

word_index = tokenizer.word_index
total_words = len(tokenizer.word_index) + 1

In [43]:
EMBED_DIM = 32
LSTM_OUT = 64
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='selu'))


In [44]:
model.compile(optimizer='adam',
                loss = tf.keras.losses.binary_crossentropy,
                metrics=['accuracy'])

model.fit(x_train_padded, y_train, epochs=20, validation_data=(x_test_padded, y_test))

Epoch 1/20
50/50 [==============================] - 7s 102ms/step - loss: -5.0775 - accuracy: 0.3288 - val_loss: -7.2434 - val_accuracy: 0.3750
Epoch 2/20
50/50 [==============================] - 6s 114ms/step - loss: -8.3299 - accuracy: 0.3406 - val_loss: -7.2434 - val_accuracy: 0.3750
Epoch 3/20
50/50 [==============================] - 2s 49ms/step - loss: -8.3299 - accuracy: 0.3406 - val_loss: -7.2434 - val_accuracy: 0.3750
Epoch 4/20
50/50 [==============================] - 2s 46ms/step - loss: -8.3299 - accuracy: 0.3406 - val_loss: -7.2434 - val_accuracy: 0.3750
Epoch 5/20
50/50 [==============================] - 2s 46ms/step - loss: -8.3299 - accuracy: 0.3406 - val_loss: -7.2434 - val_accuracy: 0.3750
Epoch 6/20
50/50 [==============================] - 2s 46ms/step - loss: -8.3299 - accuracy: 0.3406 - val_loss: -7.2434 - val_accuracy: 0.3750
Epoch 7/20
50/50 [==============================] - 4s 77ms/step - loss: -8.3299 - accuracy: 0.3406 - val_loss: -7.2434 - val_accuracy: 0.37